In [61]:
import requests

from bs4 import BeautifulSoup

baseurl = "https://data.planmap.eu/pub/"

# html = requests.get(url).content

# soup = BeautifulSoup(html,"html5lib")



# mov_container = mlist.find_all("div",class_="lister-item mode-advanced")

In [62]:
import attr

@attr.attrs
class Body():
    name = attr.ib(None)
    maps = attr.ib(None)
    
@attr.attrs
class Product():
    name = attr.ib(None)
    url = attr.ib(None)
    all_docs = attr.ib(None)
    thumb = attr.ib(None)
    thumb_url = attr.ib(None)
    

    
    

In [63]:
def fetch_address(url):
    html = requests.get(url).content
    soup = BeautifulSoup(html,"html5lib")
    return soup
    
def get_cleaned_links(soup, ignores=[]):
    found = soup.find_all(["a"], recursive=True)
    
    out = []
    for f in found:
        if list(f.children)[0] == f["href"] and f["href"] not in ignore:
            if f["href"][-1] == "/":
                name=f["href"][:-1]
            else:
                name =f["href"]
            out.append(name)
            
    return out

In [64]:
bodies = ["mars", "mercury", "moon"]
# bodies = ["mars"]
ignore =["versions/", "README.md"]

bb = []
for body in bodies:
    b = Body()
    b.name = body
    url = baseurl + body
    soup = fetch_address(url)
#     found = soup.find_all(["a"], recursive=True)
    
    maps = []
    out = get_cleaned_links(soup, ignore)
    for name in out:
        m = Product(name=name)
        maps.append(m)
#     for f in found:
#         if list(f.children)[0] == f["href"] and f["href"] not in ignore:
#             m = Map(name=f["href"][:-1])
#             out.append(m)
    
    b.maps = maps
    bb.append(b)
    
    

    


In [65]:
bb

[Body(name='mars', maps=[Product(name='PM-MAR-C-Arsinoes', url=None, all_docs=None, thumb=None, thumb_url=None), Product(name='PM-MAR-C-Crommelin', url=None, all_docs=None, thumb=None, thumb_url=None), Product(name='PM-MAR-D-Crommelin', url=None, all_docs=None, thumb=None, thumb_url=None), Product(name='PM-MAR-D-Gale_kimberley', url=None, all_docs=None, thumb=None, thumb_url=None), Product(name='PM-MAR-MS-Arsinoes', url=None, all_docs=None, thumb=None, thumb_url=None), Product(name='PM-MAR-MS-Crommelin', url=None, all_docs=None, thumb=None, thumb_url=None)]),
 Body(name='mercury', maps=[Product(name='PM-MER-C-Beethoven', url=None, all_docs=None, thumb=None, thumb_url=None), Product(name='PM-MER-C-H05', url=None, all_docs=None, thumb=None, thumb_url=None), Product(name='PM-MER-C-Rembrandt', url=None, all_docs=None, thumb=None, thumb_url=None), Product(name='PM-MER-MS-H02_3cc', url=None, all_docs=None, thumb=None, thumb_url=None), Product(name='PM-MER-MS-H05_3cc', url=None, all_docs=None

In [66]:
# compose the per map urls
for body in bb:
    for map in body.maps:
        new_addr = baseurl + body.name + "/" + map.name
        
        map.url = new_addr
        


In [67]:
# append the list of available docs
for body in bb:
    for map in body.maps:
        docurl = map.url + "/" + "document/"
        soup = fetch_address(docurl)
        links = get_cleaned_links(soup, ignore)
        map.all_docs = links
        
    

In [68]:
#select proper thumbnails
import re
for body in bb:
    for map in body.maps:
        found = None
        allpngs = []
        for doc in map.all_docs:
#             print (doc)
            ma = re.match(f"{map.name}([\_-][0-9]*)?.browse\.png", doc)
            if ma:
                found = doc
                print(f"found good thumbnail {found}")
                
            else:
                ma = re.match(f"{map.name}.*(\.browse)?\.png", doc)
                if ma:
                    allpngs.append(doc)
        
        if not found:
            print(f"Not found for {map.name}")
            if len(allpngs) > 0:
                print(f"autoassign {allpngs[0]}")
                found = allpngs[0]
            
        
        map.thumb = found         

Not found for PM-MAR-C-Arsinoes
autoassign PM-MAR-C-Arsinoes_FRT00008233_07-location.browse.png
found good thumbnail PM-MAR-C-Crommelin.browse.png
Not found for PM-MAR-D-Crommelin
Not found for PM-MAR-D-Gale_kimberley
found good thumbnail PM-MAR-MS-Arsinoes.browse.png
found good thumbnail PM-MAR-MS-Crommelin.browse.png
Not found for PM-MER-C-Beethoven
autoassign PM-MER-C-Beethoven_450mpp-0.browse.png
found good thumbnail PM-MER-C-H05.browse.png
found good thumbnail PM-MER-C-H05_02.browse.png
Not found for PM-MER-C-Rembrandt
autoassign PM-MER-C-Rembrandt_630mpp_clem.browse.png
found good thumbnail PM-MER-MS-H02_3cc.browse.png
found good thumbnail PM-MER-MS-H05_3cc.browse.png
found good thumbnail PM-MER-MS-H05_5cc.browse.png
found good thumbnail PM-MER-MS-Rembrandt.browse.png
found good thumbnail PM-MOO-C-SPAApollo.browse.png
found good thumbnail PM-MOO-MS-Copernicus.browse.png
found good thumbnail PM-MOO-MS-SPAApollo.browse.png


In [69]:
# update thumbnails urls
for body in bb:
    for map in body.maps:
        thumb = map.thumb
        if thumb:
            map.thumb_url = map.url + "/" + "document/" + thumb

In [70]:
from xml.etree import ElementTree as ET
tdattrib={"style": "background: transparent; border: transparent;  text-align: center; border:#FFF0; background:#FFF0"}

def generate_html_map_rows(map, container):
    row1 = ET.Element('tr')
    td = ET.Element('td', attrib=tdattrib)
    row1.append(td)

    td.text = map.name


    row2 = ET.Element('tr')
    td1 = ET.Element('td', attrib=tdattrib)
    row2.append(td1)


    link = ET.Element("a", attrib={"href": map.url})
    img = ET.Element("img", attrib={"alt":f"To datahub ({map.name})", "src": map.thumb_url, "width":"600", "style": "border-radius:2%;  object-fit: contain;"})
    link.append(img)
    td1.append(link)
    container.append(row1)
    container.append(row2)

In [71]:
# geenrate html
div = ET.Element("div")
table = ET.Element('table', attrib={"style": "background: transparent; border: transparent;  text-align: center;", "class":"geomaps"})
tbody = ET.Element('tbody')
table.append(tbody)

def generate_maps_table(maps, container, title=None, name_filter_regex=None, invert_filter=False):
    
    div = ET.Element("div")
    table = ET.Element('table', attrib={"style": "background: transparent; border: transparent;  text-align: center;", "class":"geomaps"})
    tbody = ET.Element('tbody')
    table.append(tbody)
    
    
    if title:
        p = ET.Element("h2", attrib={"class":"rtecenter"})
        st = ET.Element("strong")
        p.append(st)
        st.text = title
        container.append(p)
        
    div.append(table)
        
    for map in maps:
        if name_filter_regex:
            match = False
            if re.match(name_filter_regex, map.name):
                match=True
            if invert_filter:
                match = not match
            
        
            if match:
                print(f"adding {map.name}")
                generate_html_map_rows(map, tbody)
            
        else:
            generate_html_map_rows(map, tbody)
        
        
    container.append(div)


        


In [72]:
div = ET.Element("div")
for body in bb:
    generate_maps_table(body.maps, div, body.name.title(),name_filter_regex=".*\-C\-.*")
ET.ElementTree(div).write("spectral.html")

adding PM-MAR-C-Arsinoes
adding PM-MAR-C-Crommelin
adding PM-MER-C-Beethoven
adding PM-MER-C-H05
adding PM-MER-C-Rembrandt
adding PM-MOO-C-SPAApollo


In [73]:
div = ET.Element("div")
for body in bb:
    generate_maps_table(body.maps, div, body.name.title(), name_filter_regex=".*\-MS\-.*")
ET.ElementTree(div).write("geological.html")

adding PM-MAR-MS-Arsinoes
adding PM-MAR-MS-Crommelin
adding PM-MER-MS-H02_3cc
adding PM-MER-MS-H05_3cc
adding PM-MER-MS-H05_5cc
adding PM-MER-MS-Rembrandt
adding PM-MOO-MS-Copernicus
adding PM-MOO-MS-SPAApollo


In [74]:
for map in body.maps:
    print(map.name)
    print(map.thumb_url)
    print(map.url)

PM-MOO-C-SPAApollo
https://data.planmap.eu/pub/moon/PM-MOO-C-SPAApollo/document/PM-MOO-C-SPAApollo.browse.png
https://data.planmap.eu/pub/moon/PM-MOO-C-SPAApollo
PM-MOO-MS-Copernicus
https://data.planmap.eu/pub/moon/PM-MOO-MS-Copernicus/document/PM-MOO-MS-Copernicus.browse.png
https://data.planmap.eu/pub/moon/PM-MOO-MS-Copernicus
PM-MOO-MS-SPAApollo
https://data.planmap.eu/pub/moon/PM-MOO-MS-SPAApollo/document/PM-MOO-MS-SPAApollo.browse.png
https://data.planmap.eu/pub/moon/PM-MOO-MS-SPAApollo


In [75]:
import sys


html = ET.Element('html')
body = ET.Element('body')
html.append(body)
div = ET.Element('div', attrib={'class': 'foo'})
body.append(div)
span = ET.Element('span', attrib={'class': 'bar'})
div.append(span)
span.text = "Hello World"


# python 3
ET.ElementTree(html).write(sys.stdout, encoding='unicode',
                             method='html')

<html><body><div class="foo"><span class="bar">Hello World</span></div></body></html>